In [1]:
import torch
import torch.nn as nn
import numpy as np
import pickle
from fastai.text.all import *

In [2]:
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
device

'cpu'

In [129]:
def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / np.sqrt(dim_k)
    if mask is not None:
        if scores.shape[1] == mask.shape[1]:
            scores = scores.masked_fill(mask == 0, float("-inf"))
        else:
            mask = torch.tril(torch.ones(scores.shape[1], scores.shape[1])).unsqueeze(0).to(device)
            scores = scores.masked_fill(mask == 0, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights.bmm(value)

class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim, vocab_size):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)
        self.mask = torch.tril(torch.ones(vocab_size, vocab_size)).unsqueeze(0).to(device)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state), self.mask)
        return attn_outputs

class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim, 72) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # Apply layer normalization and then copy input into query, key, value
        hidden_state = self.layer_norm_1(x)
        # Apply attention with a skip connection
        x = x + self.attention(hidden_state)
        # Apply feed-forward layer with a skip connection
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size, 
                                             config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # Create position IDs for input sequence
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0).to(device)
        # Create token and position embeddings
        token_embeddings = self.token_embeddings(input_ids)
        
        position_embeddings = self.position_embeddings(position_ids)
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config) 
                                     for _ in range(config.num_hidden_layers)])

    def forward(self, x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

class ShellTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        
    def forward(self, x):
        #print(x)
        x = self.encoder(x)#[:, 0, :] # select hidden state of [CLS] token
        #print(x)
        x = self.dropout(x)
        x = self.classifier(x)
        #print(x)
        return x

In [298]:
with open('/home/chris/University/gnn_project/dataset', 'rb') as fp:
    _ = pickle.load(fp)

In [318]:
_[0:10]

['nmap\n',
 'nmap -v 10.1.26.4\n',
 'nmap -v 10.1.26.9\n',
 'ssh --help\n',
 'ssh 10.1.26.9\n',
 'ssh 10.1.26.9 admin/123456\n',
 'ssh --help\n',
 'ssh 10.1.26.9\n',
 'ssh -l admin 10.1.26.9\n',
 'ssh admin@admin 10.1.26.9\n']

In [320]:
# Given array of text elements
texts = _

# Create a folder to store the text files
import os
folder_path = '/home/chris/University/gnn_project/data/'
os.makedirs(folder_path, exist_ok=True)

# Write each text element to a separate file
for i, text in enumerate(texts):
    file_path = os.path.join(folder_path, f'text_{i}.txt')
    with open(file_path, 'w') as file:
        file.write(text)


In [299]:
txts = L(_)

In [332]:
class MyTokenizer(Transform):
    def setups(self, items):
        path = untar_data(URLs.IMDB)
        self.tok =  Tokenizer.from_folder(path)
        self.tok.setup(items)
        
    def encodes(self, txts):
        with open(txts, 'r') as file:
            content = file.read()
        return self.tok(content)
    
    def decodes(self, encoded):
        return self.tok.decode(encoded)
            
class MyNumerizer(Transform):
    def setups(self, items):
        self.num = Numericalize()
        self.num.setup(items)
        self.vocab = self.num.vocab

    def encodes(self, toks):
        return self.num(toks)
    
    def decodes(self, encoded):
        return self.num.decode(encoded)
    
    
    
#path_test = untar_data(URLs.IMDB)
path_test = '/home/chris/University/gnn_project/'
tfms = [[MyTokenizer(),MyNumerizer()]]
files = get_text_files(path_test, folders = ['data'])
#files = get_text_files(path_test, folders = ['train'])
print(files[0:5])
#files = get_text_files(path, folders = ['train'])
dsets = Datasets(files[:], tfms)
print(dsets[0:5])
dls = dsets.dataloaders(dl_type=LMDataLoader, bs=4)

dls.show_batch(max_n=10)


[Path('/home/chris/University/gnn_project/data/text_173923.txt'), Path('/home/chris/University/gnn_project/data/text_97621.txt'), Path('/home/chris/University/gnn_project/data/text_51016.txt'), Path('/home/chris/University/gnn_project/data/text_198692.txt'), Path('/home/chris/University/gnn_project/data/text_117671.txt')]
[(TensorText([2]),), (TensorText([ 2, 79]),), (TensorText([ 2, 10]),), (TensorText([   2,  379,   75, 4433,  138,   12,  210,  491, 7255]),), (TensorText([   2,   17, 3725]),)]


,text,text_
0,xxbos vi out xxbos echo 100 > / sys / class / backlight / intel_backlight / brightness xxbos nano xxmaj controllers / xxunk xxbos ls xxbos sudo nano / etc / network / interfaces xxbos help xxbos sudo service collectd start xxbos [ goalador@gatanda hhneuauf.de]$ bash : [ goalador@gatanda : command not found xxbos clear xxbos # xxunk xxbos tree xxbos vim .xresources xxbos vim palind.rb xxbos chmod xxunk xxmaj public /,vi out xxbos echo 100 > / sys / class / backlight / intel_backlight / brightness xxbos nano xxmaj controllers / xxunk xxbos ls xxbos sudo nano / etc / network / interfaces xxbos help xxbos sudo service collectd start xxbos [ goalador@gatanda hhneuauf.de]$ bash : [ goalador@gatanda : command not found xxbos clear xxbos # xxunk xxbos tree xxbos vim .xresources xxbos vim palind.rb xxbos chmod xxunk xxmaj public / xxbos
1,"-n xxunk xxbos ll xxbos # xxunk xxbos python xxbos cm --pkg auv_map xxbos 0mt+olwjfsja072upuhe7gwdlexqh0l59q4ohog+zjke1u74br9wo6kiv54iqgvs xxbos j=0;for i in * .jpg ; do printf -v new "" % 02d "" "" $ ( ( + + j ) ) "" ; mv -v -- "" $ i "" 2 - hh - xxunk "" ; done xxbos ls xxbos ls -al xxbos bzr push lp:~williams - jayson / + junk / bashprefs","xxunk xxbos ll xxbos # xxunk xxbos python xxbos cm --pkg auv_map xxbos 0mt+olwjfsja072upuhe7gwdlexqh0l59q4ohog+zjke1u74br9wo6kiv54iqgvs xxbos j=0;for i in * .jpg ; do printf -v new "" % 02d "" "" $ ( ( + + j ) ) "" ; mv -v -- "" $ i "" 2 - hh - xxunk "" ; done xxbos ls xxbos ls -al xxbos bzr push lp:~williams - jayson / + junk / bashprefs xxbos"
2,0 xxbos pintos -j 1 -v -k --bochs --filesys - size=2 -p tests / vm / page - parallel -a page - parallel -p tests / vm / child - linear -a child - linear --swap - size=4 -- -q -f -ul=10 run page - parallel xxbos ls xxbos vim manage.py xxbos . / relacao 2 2 xxbos ls xxbos du . / download_img / -d 1 xxbos show options xxbos i,xxbos pintos -j 1 -v -k --bochs --filesys - size=2 -p tests / vm / page - parallel -a page - parallel -p tests / vm / child - linear -a child - linear --swap - size=4 -- -q -f -ul=10 run page - parallel xxbos ls xxbos vim manage.py xxbos . / relacao 2 2 xxbos ls xxbos du . / download_img / -d 1 xxbos show options xxbos i d
3,temp / kodi.log xxbos [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ bash : [ goalador[goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ bash : [ goalador@gatanda : command not found xxbos make all xxbos rm trabalho xxbos trabalho xxbos top xxbos pintos -v -k -t 60 --qemu --filesys - size=2 -p tests / vm / page - parallel -a page - parallel -p tests / vm /,/ kodi.log xxbos [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ bash : [ goalador[goalador@gatanda hhneuauf.de]$ [ goalador@gatanda hhneuauf.de]$ bash : [ goalador@gatanda : command not found xxbos make all xxbos rm trabalho xxbos trabalho xxbos top xxbos pintos -v -k -t 60 --qemu --filesys - size=2 -p tests / vm / page - parallel -a page - parallel -p tests / vm / child


In [333]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained('bert-base-uncased')

config.vocab_size = len(dls.vocab)
config.num_labels = len(dls.vocab)
config.hidden_size = 132
transformer = ShellTransformer(config)

In [334]:
model = transformer

model.to(device)

dls.to(device)

learn = Learner(
    dls, 
    model, 
    loss_func=CrossEntropyLossFlat(), 
    metrics=[accuracy]
)

learn.fit_one_cycle(1, 1e-3)

#learn.export('mymodel.pkl')

epoch,train_loss,valid_loss,accuracy,time
0,3.600421,None,None,43:39


/home/chris/University/GNN/project/transformer_env/lib/python3.9/site-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [338]:
path = untar_data(URLs.IMDB)
tok = Tokenizer.from_folder(path)
tok.setup(txts)
toks = txts.map(tok)

In [309]:
num = Numericalize()
num.setup(toks)

In [339]:
mytokenizer = Pipeline([tok,num])

# Define a function for text generation
def generate_text(model, starting_text, max_length=10):
    token_ids = mytokenizer(starting_text).to(device)
    input_ids = torch.tensor(token_ids).unsqueeze(0).to(device)  
    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(input_ids)
            logits = outputs[:, -1, :]  
            next_token_id = torch.argmax(logits, dim=-1)
            token_ids = torch.cat((token_ids, next_token_id),dim=0)
            input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0).to(device)], dim=-1)
            if mytokenizer.decode(next_token_id) == 'xxboss':
                break
    return mytokenizer.decode(token_ids)

# Generate text
generated_text = generate_text(learn.model, "ssh --help")
print(generated_text)


/tmp/ipykernel_4491/600091294.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(token_ids).unsqueeze(0).to(device)


xxbos ssh --help xxbos ls xxbos ls xxbos ls xxbos ls xxbos ls


In [336]:
model(torch.tensor([[2, 2, 150]]))

tensor([[[ 3.3905, -6.9732,  2.0654,  ..., -6.2189, -6.3091, -6.3464],
         [ 3.3691, -6.6982,  2.0184,  ..., -6.1612, -6.1415, -6.1101],
         [ 3.9643, -6.8780,  3.5139,  ..., -6.0036, -5.1728, -5.2569]]],
       grad_fn=<ViewBackward0>)